In [8]:
# Initialize OK
from client.api.notebook import Notebook
ok = Notebook('hw3.ok')

Assignment: hw3
OK, version v1.13.11



# Homework 3: SQL

## Due Date: Monday, Feb 10th, 11:59PM

## Collaboration Policy

Data science is a collaborative activity. While you may talk with others about
the homework, we ask that you **write your solutions individually**. If you do
discuss the assignments with others please **include their names** below.

**Collaborators**: *list  collaborators here*

## Grading

Grading is broken down into autograded answers and free response. 

For autograded answers, the results of your code are compared to provided and/or hidden tests. **For autograded probability questions, the provided tests will only check that your answer is within a reasonable range.**

### Please make sure to save regularly; do not rely on autosave to save your work for you!

**Caution:** Be careful with large SQL queries.  You may need to reboot your Jupyter Hub instance if it stops responding.

## Scoring Breakdown

|Question|Points|
|---|---|
|Q1 | 2 |
|Q2 | 2 |
|Q3 | 1 |
|Q4 | 2 |
|Q5 | 3 |
|Q6 | 3 |
|Q7 | 2 |
|Q8 | 4 |
|Q9 | 4 |


## Introduction

In this homework, you will have a chance to test your SQL knowledge by writing some SQL queries. We're going to be exploring the publicly available [IMDB](https://www.imdb.com) datasets with data about films and the people involved in their creation.

In [9]:
import numpy as np
import pandas as pd
import sqlite3

In [10]:
import matplotlib.pyplot as plt
import plotly.offline as py
import cufflinks as cf

## Setup

We have constructed an IMDB database snapshot from [IMDB data](https://www.imdb.com/interfaces/).  This is a very large file so we have mounted it in a shared filesystem on Jupyter Hub.  

We will use `run_sql` function in `ds100_utils`.  This allows us to update the database query code should we run into performance issues on the jupyter hub.

In [11]:
r = ['']

In [12]:
from ds100_utils import run_sql
help(run_sql)

Help on function run_sql in module ds100_utils:

run_sql(query, limit=1000)
    Takes a query string and returns a dataframe. The limit argument
    specifies the maximum number of rows to return in response to a query.



## Getting to Know Your Data

Now that we have downloaded our SQLite database, we want to make sure we understand the **schema** of our data before we start writing queries. The following query returns the tables stored in the database.

In [13]:
for _, row in run_sql("SELECT sql FROM sqlite_master WHERE type='table' ").iterrows():
    print(row['sql'], '\n')

CREATE TABLE names (
    nid CHAR(10) PRIMARY KEY,
    name TEXT,
    birth_year INTEGER,
    death_year INTEGER
) 

CREATE TABLE titles (
    tid CHAR(10) PRIMARY KEY,
    type TEXT,
    title TEXT,
    is_adult TEXT,
    start_year INTEGER,
    end_year INTEGER,
    runtime_minutes INTEGER
) 

CREATE TABLE roles (
    tid CHAR(10) NOT NULL,
    nid CHAR(10) NOT NULL,
    category TEXT,
    ordering INTEGER,
    FOREIGN KEY (tid) REFERENCES titles(tid),
    FOREIGN KEY (nid) REFERENCES names(nid)
) 

CREATE TABLE ratings (
    tid CHAR(10) NOT NULL,
    avg_rating REAL,
    num_votes INTEGER,
    FOREIGN KEY (tid) REFERENCES titles(tid)
) 



We have four tables.  Notice that the `names` and `titles` tables have primary keys that the `roles` and `ratings` tables reference. 

Documentation for the schema can be found [here](https://www.imdb.com/interfaces/). Note we have renamed `principals` to `roles`.  We have also slightly shorted or simplified several of the column names.

To get a better understanding of the values we will look at 10 records from each table using the `run_sql` command.  

In [25]:
run_sql("""
    SELECT * FROM names LIMIT 10
""")

,nid,name,birth_year,death_year
0,nm0000001,Fred Astaire,1899,1987.0
1,nm0000002,Lauren Bacall,1924,2014.0
2,nm0000003,Brigitte Bardot,1934,NaN
3,nm0000004,John Belushi,1949,1982.0
4,nm0000005,Ingmar Bergman,1918,2007.0
5,nm0000006,Ingrid Bergman,1915,1982.0
6,nm0000007,Humphrey Bogart,1899,1957.0
7,nm0000008,Marlon Brando,1924,2004.0
8,nm0000009,Richard Burton,1925,1984.0
9,nm0000010,James Cagney,1899,1986.0


In [26]:
run_sql("""SELECT AVG(birth_year) AS avg_birth FROM names GROUP BY death_year HAVING birth_year > 0""")

,avg_birth
0,1967.359817
1,1868.000000
2,59.000000
3,1932.000000
4,4.000000
...,...
411,1940.214609
412,1940.695956
413,1940.830935
414,1942.850384


In [27]:
run_sql("""SELECT COUNT(*) FROM names GROUP BY name LIMIT 10""")

,COUNT(*)
0,1
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,1


In [28]:
run_sql("""
    SELECT * FROM titles LIMIT 10
""")

,tid,type,title,is_adult,start_year,end_year,runtime_minutes
0,tt0000009,movie,Miss Jerry,0,1894,None,45.0
1,tt0000147,movie,The Corbett-Fitzsimmons Fight,0,1897,None,20.0
2,tt0000335,movie,Soldiers of the Cross,0,1900,None,NaN
3,tt0000502,movie,Bohemios,0,1905,None,100.0
4,tt0000574,movie,The Story of the Kelly Gang,0,1906,None,70.0
5,tt0000615,movie,Robbery Under Arms,0,1907,None,NaN
6,tt0000630,movie,Hamlet,0,1908,None,NaN
7,tt0000675,movie,Don Quijote,0,1908,None,NaN
8,tt0000676,movie,Don Álvaro o la fuerza del sino,0,1908,None,NaN
9,tt0000679,movie,The Fairylogue and Radio-Plays,0,1908,None,120.0


In [29]:
run_sql("""
    SELECT * FROM roles LIMIT 10
""")

,tid,nid,category,ordering
0,tt0000009,nm0063086,actress,1
1,tt0000009,nm0183823,actor,2
2,tt0000009,nm1309758,actor,3
3,tt0000009,nm0085156,director,4
4,tt0000147,nm0179163,self,1
5,tt0000147,nm0280615,self,2
6,tt0000147,nm4082222,self,3
7,tt0000147,nm4081458,self,4
8,tt0000147,nm0714557,director,5
9,tt0000147,nm0103755,producer,6


In [30]:
run_sql("""
    SELECT * FROM ratings LIMIT 10
""")

,tid,avg_rating,num_votes
0,tt0000009,5.4,86
1,tt0000147,5.2,324
2,tt0000335,6.1,40
3,tt0000502,4.4,5
4,tt0000574,6.1,553
5,tt0000615,4.8,14
6,tt0000630,3.4,11
7,tt0000675,4.2,9
8,tt0000676,3.6,8
9,tt0000679,4.8,33


### Important Schema Information

* `names` contains information about individuals involved in films.
* `titles` contains information about the films.
* `roles` relates individuals to titles
* `ratings` contains information about the ratings associated with each film. 
* `names.nid` is a Primary Key
* `titles.tid` is a Primary Key
* `roles.nid` and `roles.tid` are foreign keys that point to `names.nid` and `titles.tid`, respectively.



---

## Question 1

How far back does our data go? Does it only include recent data, or do we have information about older movies and movie stars as well? 

List the **10 oldest films** by `start_year` and then `title` both in **ascending** order.  Do not include films where the `start_year` is `NULL`.  The output should contain the `start_year`, `type`, and `title`.


<!--
BEGIN QUESTION
name: q1
manual: false
points: 2
-->

In [31]:
oldestMovieQuery = """SELECT start_year, type, title FROM titles WHERE start_year != 'None' and type = 'movie' ORDER BY start_year, title LIMIT 10"""
oldestMovieDf = run_sql(oldestMovieQuery)
oldestMovieDf

,start_year,type,title
0,1894,movie,Miss Jerry
1,1897,movie,Reproduction of the Corbett and Fitzsimmons Fight
2,1897,movie,The Corbett-Fitzsimmons Fight
3,1898,movie,69th Regiment Passing in Review
4,1898,movie,A Rua Augusta em Dia de Festa
5,1898,movie,A Very Laughable Mixup
6,1898,movie,Bombardment of Matanzas
7,1898,movie,Branding Cattle
8,1898,movie,"Buck Dance, Ute Indians"
9,1898,movie,Cake Walk


In [32]:
ok.grade("q1");

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 2
    Failed: 0
[ooooooooook] 100.0% passed



---

## Question 2

Next, let's calculate the distribution of films by year. Write a query that returns the **total** films for each `start_year` in the `titles` table as `total`.  Keep in mind that some entries may not have a `start_year` year listed -- you should filter those out.  Order your final results by the `start_year`.

The first 5 lines of the table should look like the following (but you should compute the entire table).
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>start_year</th>
      <th>total</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>1894</td>
      <td>1</td>
    </tr>
    <tr>
      <th>1</th>
      <td>1897</td>
      <td>2</td>
    </tr>
    <tr>
      <th>2</th>
      <td>1898</td>
      <td>26</td>
    </tr>
    <tr>
      <th>3</th>
      <td>1899</td>
      <td>12</td>
    </tr>
    <tr>
      <th>4</th>
      <td>1900</td>
      <td>14</td>
    </tr>
  </tbody>
</table>


<!--
BEGIN QUESTION
name: q2
manual: false
points: 2
-->

In [33]:
yearDistQuery = """SELECT start_year, COUNT(title) AS total FROM titles WHERE start_year != 'None' and type = 'movie' GROUP BY start_year"""
yearDistDf = run_sql(yearDistQuery)
yearDistDf

,start_year,total
0,1894,1
1,1897,2
2,1898,26
3,1899,12
4,1900,14
...,...,...
128,2024,12
129,2025,2
130,2026,5
131,2027,2


In [34]:
ok.grade("q2");

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 1
    Failed: 0
[ooooooooook] 100.0% passed



The following should generate an interesting plot of the number of films that premiered each year. Notice there is a dip between the 1920s and late 1940s. Why might that be? *This question is rhetorical; you do not need to write your answer anywhere.*

In [35]:
yearDistDf.iplot(kind="bar", x="start_year", y="total", 
                 xTitle="Start Year", yTitle="Count", asFigure=True)

---

## Question 3

What are the different kinds of `types` included in the `titles` table? Write a query to find out all the unique `types` of films using the `DISTINCT` keyword.  (**You may not use `GROUP BY`.**)

<!--
BEGIN QUESTION
name: q3
manual: false
points: 1
-->

In [36]:
typeQuery = """SELECT DISTINCT(type) FROM titles"""
typeDf = run_sql(typeQuery)
typeDf

,type
0,movie
1,tvMovie


In [37]:
ok.grade("q3");

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 2
    Failed: 0
[ooooooooook] 100.0% passed



---

## Question 4

Before we proceed we want to get a better picture of the kinds of jobs that exist.  To do this examine the `roles` table by computing the number of records with each job `category`.  Present the results in descending order by the total counts.

The top of your table should look like this:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>category</th>
      <th>total</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>actor</td>
      <td>8739100</td>
    </tr>
    <tr>
      <th>1</th>
      <td>actress</td>
      <td>6541392</td>
    </tr>
    <tr>
      <th>2</th>
      <td>self</td>
      <td>6334197</td>
    </tr>
  </tbody>
</table>

However, there should be more rows.


<!--
BEGIN QUESTION
name: q4
manual: false
points: 2
-->

In [38]:
roleCategoriesQuery = """SELECT category, COUNT(category) AS total FROM roles GROUP BY category ORDER BY -total"""
roleCategoriesDf = run_sql(roleCategoriesQuery)
roleCategoriesDf

,category,total
0,actor,1286695
1,actress,779314
2,director,614814
3,writer,510314
4,producer,391252
5,cinematographer,351163
6,composer,297783
7,self,216523
8,editor,210068
9,production_designer,46419


In [39]:
ok.grade("q4");

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 1
    Failed: 0
[ooooooooook] 100.0% passed



If we computed the results correctly we should see a nice horizontal bar chart of the counts per category below:

In [40]:
roleCategoriesDf.iplot(kind="barh", x ="category", y = "total", xTitle="Count", asFigure=True)

---


## Question 5

Now that we have a better sense of the basics of our data, we can ask some more interesting questions.  In this question, we will explore which 10 **actors** or **actresses** were in the most **movies**.  Construct a table consisting of their name (as `name`) and the number of movies in which they performed as `total` and order the results by the total in descending order and break ties by ordering by `name` in ascending order.

***The query should take < 2 minutes to run.***

*Hint: Google the top of the list and see if it makes sense.*

*Hint: If you want to include a non-aggregate field in the* `SELECT` *clause, it must also be included in the* `GROUP BY` *clause.*

Your result should look something like the following but without `????`:
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>name</th>
      <th>total</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>????</td>
      <td>809</td>
    </tr>
    <tr>
      <th>1</th>
      <td>????</td>
      <td>584</td>
    </tr>
    <tr>
      <th>2</th>
      <td>????</td>
      <td>565</td>
    </tr>
    <tr>
      <th>3</th>
      <td>????</td>
      <td>507</td>
    </tr>
    <tr>
      <th>4</th>
      <td>????</td>
      <td>411</td>
    </tr>
    <tr>
      <th>5</th>
      <td>????</td>
      <td>409</td>
    </tr>
    <tr>
      <th>6</th>
      <td>????</td>
      <td>391</td>
    </tr>
    <tr>
      <th>7</th>
      <td>????</td>
      <td>388</td>
    </tr>
    <tr>
      <th>8</th>
      <td>????</td>
      <td>382</td>
    </tr>
    <tr>
      <th>9</th>
      <td>????</td>
      <td>360</td>
    </tr>
  </tbody>
</table>


<!--
BEGIN QUESTION
name: q5
manual: false
points: 3
-->

In [41]:
prolificPerformersQuery = """SELECT n.name, COUNT(n.name) as total FROM names as n, roles as r WHERE n.nid = r.nid and (r.category = "actor" or r.category = "actress") GROUP BY n.name ORDER BY -total, n.name LIMIT 10"""
prolificPerformersDf = run_sql(prolificPerformersQuery)
prolificPerformersDf

,name,total
0,Brahmanandam,809
1,Adoor Bhasi,584
2,Matsunosuke Onoe,565
3,Eddie Garcia,509
4,Sung-il Shin,411
5,Prem Nazir,409
6,Paquito Diaz,393
7,Masayoshi Nogami,388
8,Mammootty,382
9,Bahadur,360


In [42]:
ok.grade("q5");

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 2
    Failed: 0
[ooooooooook] 100.0% passed



---

## Question 6


We also have average `ratings` data consisting of the `num_votes` and the `average_rating` for each title.  Let's first see how many **movies** have ratings.  Construct the following table:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>has_rating</th>
      <th>total</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>no</td>
      <td>????</td>
    </tr>
    <tr>
      <th>1</th>
      <td>yes</td>
      <td>????</td>
    </tr>
  </tbody>
</table>

Where `????` is replaced with the correct values. The row with `no` should have the count for many movies **do not** have ratings, and the row with `yes` should have the count of how many movies **do** have ratings.

You will need to use a `LEFT JOIN`. You may also consider using a `CASE WHEN ... IS NOT NULL THEN 'yes" ... ELSE ... END` statement. `CASE` statements are the SQL-equivalent of Python `if... elif... else` statements. Take a look at the following links for help with `CASE` statements:
- https://mode.com/sql-tutorial/sql-case/
- https://www.w3schools.com/sql/sql_ref_case.asp


<!--
BEGIN QUESTION
name: q6
manual: false
points: 3
-->

In [43]:
missingRatingsQuery = """SELECT CASE WHEN avg_rating IS NULL THEN 'no' ELSE 'yes' END AS has_rating, COUNT(*) AS total FROM titles as t LEFT JOIN ratings as r ON t.tid = r.tid WHERE t.type = 'movie' GROUP BY has_rating"""
missingRatingsDf = run_sql(missingRatingsQuery)
missingRatingsDf

,has_rating,total
0,no,298361
1,yes,244114


In [44]:
ok.grade("q6");

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 3
    Failed: 0
[ooooooooook] 100.0% passed



---

## Question 7


We also have average `ratings` data consisting of the `num_votes` and the `average_rating` for each title.  Which films have the most ratings?  Make a table consisting of the `title`, `num_votes`, and `avg_rating` for the 10 films that have the highest number of ratings.  Sort the result in descending order by the number of votes.


<!--
BEGIN QUESTION
name: q7
manual: false
points: 2
-->

In [45]:
popularVotesQuery = """SELECT t.title, r.num_votes, r.avg_rating FROM ratings as r, titles as t WHERE r.tid = t.tid ORDER BY -r.num_votes LIMIT 10"""
popularVotesDf = run_sql(popularVotesQuery)
popularVotesDf

,title,num_votes,avg_rating
0,The Shawshank Redemption,2185370,9.3
1,The Dark Knight,2168354,9.0
2,Inception,1917188,8.8
3,Fight Club,1745578,8.8
4,Pulp Fiction,1716857,8.9
5,Forrest Gump,1685110,8.8
6,The Matrix,1573548,8.7
7,The Lord of the Rings: The Fellowship of the Ring,1566499,8.8
8,The Lord of the Rings: The Return of the King,1554618,8.9
9,The Godfather,1505543,9.2


In [46]:
ok.grade("q7");

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 2
    Failed: 0
[ooooooooook] 100.0% passed



---

## Question 8


How does film length relate to ratings?  To answer this question we want to bin films by length and compute the average of the average ratings within each length bin. We will group movies by 10-minute increments -- that is, one bin for movies \[0, 10) minutes long, another for \[10, 20) minutes, another for \[20, 30) minutes, and so on. Use the following code snippet to help construct 10-minute bins: 

```
ROUND(runtimeMinutes / 10.0 + 0.5) * 10 AS runtime_bin
```

Construct a table containing the **runtime_bin**, the **average** of the **average ratings** (as `avg_rating`), the **average number of votes** (as `avg_num_votes`), and the number of `titles` in that **runtime_bin** (as `total`).  Only include movies with **at least 10000 votes**.  Order the final results by the value of `runtime_bin`.



<!--
BEGIN QUESTION
name: q8
manual: false
points: 4
-->

In [47]:
runtimeRatingsQuery = """SELECT ROUND(runtime_minutes / 10.0 + 0.5) * 10
AS runtime_bin,
AVG(avg_rating) AS avg_rating,
AVG(num_votes) AS avg_num_votes,
COUNT(title) AS total
FROM ratings as r
INNER JOIN titles as t
ON r.tid = t.tid
WHERE r.num_votes >= 10000
AND t.type = 'movie'
GROUP BY runtime_bin
ORDER BY runtime_bin"""
runtimeRatingsDf = run_sql(runtimeRatingsQuery)
runtimeRatingsDf

,runtime_bin,avg_rating,avg_num_votes,total
0,50.0,7.575000,22100.750000,4
1,70.0,7.531579,39783.210526,19
2,80.0,6.698291,39399.769231,117
3,90.0,6.174584,50916.119952,842
4,100.0,6.341646,55819.528662,2041
5,110.0,6.578053,72540.878967,1859
6,120.0,6.801418,87223.404255,1410
7,130.0,7.030115,113127.282759,870
8,140.0,7.174187,135497.653920,523
9,150.0,7.325514,181130.588477,243


In [48]:
ok.grade("q8");

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 4
    Failed: 0
[ooooooooook] 100.0% passed



If your SQL query is correct you should get some intesting plots below.  This might explain why directors keep going a particular direction with film lengths.  

In [49]:
runtimeRatingsDf.iplot(x="runtime_bin", y="avg_rating", asFigure=True)

In [50]:
runtimeRatingsDf.iplot(x="runtime_bin", y="avg_num_votes", asFigure=True)

In [51]:
runtimeRatingsDf.iplot(kind="bar", x="runtime_bin", y="total", asFigure=True)

---

## Question 9


Which **actors** and **actresses** have the highest average ratings across all the **movies** in which they star?  

Construct a table consisting of the **actor's/actress' name**  (as `name`) and their **average_rating** (as `avg_rating`) computed by rescaling their individual performance ratings:

$$
\frac{\sum_m \text{avg_rating}[m] * \text{num_votes}[m]}{\sum_m \text{num_votes}[m]}
$$

Note that if an actor/actress has multiple `role` listings for a film then that film will have a bigger impact in the overall average (this is desired).

Only consider ratings where there are **at least 1000** votes and only consider actors/actresses that have **at least 20 rated performances**. Present the actors/actresses with the top 10 `avg_rating` in descending order and break ties using the actor's/actress' name.


The results should look something like this but without the `????`:
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>name</th>
      <th>avg_rating</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>????</td>
      <td>8.477808</td>
    </tr>
    <tr>
      <th>1</th>
      <td>????</td>
      <td>8.410496</td>
    </tr>
    <tr>
      <th>2</th>
      <td>????</td>
      <td>8.396275</td>
    </tr>
    <tr>
      <th>3</th>
      <td>????</td>
      <td>8.365583</td>
    </tr>
    <tr>
      <th>4</th>
      <td>????</td>
      <td>8.355513</td>
    </tr>
    <tr>
      <th>5</th>
      <td>????</td>
      <td>8.352385</td>
    </tr>
    <tr>
      <th>6</th>
      <td>????</td>
      <td>8.333354</td>
    </tr>
    <tr>
      <th>7</th>
      <td>????</td>
      <td>8.330661</td>
    </tr>
    <tr>
      <th>8</th>
      <td>????</td>
      <td>8.269803</td>
    </tr>
    <tr>
      <th>9</th>
      <td>????</td>
      <td>8.265962</td>
    </tr>
  </tbody>
</table>


<!--
BEGIN QUESTION
name: q9
manual: false
points: 4
-->

In [52]:
topRatedPerformerQuery = """SELECT n.name, (SUM(r.avg_rating * r.num_votes)/SUM(r.num_votes)) AS avg_rating 
FROM names as n, roles, titles AS t, ratings AS r
WHERE roles.nid = n.nid
AND roles.tid = t.tid
AND r.tid = t.tid
AND t.type = 'movie'
AND num_votes >= 1000
AND (roles.category = 'actress' OR roles.category = 'actor')
GROUP BY n.nid 
HAVING COUNT(r.avg_rating) >= 20 
ORDER BY avg_rating DESC, name
LIMIT 10"""
topRatedPerformerDf = run_sql(topRatedPerformerQuery)
topRatedPerformerDf

,name,avg_rating
0,Marlon Brando,8.477808
1,Lee J. Cobb,8.410496
2,Alec Guinness,8.396275
3,Diane Keaton,8.365583
4,Elijah Wood,8.355513
5,Lionel Barrymore,8.352385
6,Henry Fonda,8.333354
7,Edward Furlong,8.330661
8,Oliver Reed,8.269803
9,Toshirô Mifune,8.265962


In [53]:
ok.grade("q9");

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 2
    Failed: 0
[ooooooooook] 100.0% passed



## THE END!

# Submit
Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output.
**Please save before submitting!**

In [ ]:
# Save your notebook first, then run this cell to submit.
ok.submit()